In [1]:
import datetime
import json
import os

import pandas as pd
import tinvest
import yfinance as yf
from pytz import timezone
from tqdm.notebook import tqdm

In [2]:
begin = "2016-01-01"
end = "2022-01-01"

In [3]:
stocks = [
    "FESH.ME",
    "APTK.ME",
    "RASP.ME",
    "NKNCP.ME",
    "MVID.ME",
    "BELU.ME",
    "PHOR.ME",
    "SNGSP.ME",
    "SELG.ME",
    "PIKK.ME",
    "YNDX.ME",
]
indexes = ["IMOEX.ME", "BZ=F", "GLD"]
currencies = ["RUB=X"]

In [4]:
path = "./data/{}.csv"
for stock in tqdm(stocks + currencies + indexes):
    spath = path.format(stock)
    if os.path.exists(spath):
        continue
    df = yf.download(stock, begin, end)
    df.to_csv(spath)

  0%|          | 0/15 [00:00<?, ?it/s]

In [5]:
with open(os.path.expanduser('~/.tcs_token'), 'r') as f:
    TOKEN = f.read().strip()
tz = timezone("Europe/Moscow")
client = tinvest.SyncClient(TOKEN, use_sandbox=True)
bonds = client.get_market_bonds().payload.instruments
bonds = pd.DataFrame([json.loads(x.json()) for x in bonds])
curr = client.get_market_currencies().payload.instruments
curr = pd.DataFrame([json.loads(x.json()) for x in curr])

In [6]:
def get_max_duration(interval):
    if interval == tinvest.schemas.CandleResolution.min1:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.min2:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.min3:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.min5:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.min10:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.min15:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.min30:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.hour:
        return datetime.timedelta(days=7)
    elif interval == tinvest.schemas.CandleResolution.day:
        return datetime.timedelta(days=365)
    elif interval == tinvest.schemas.CandleResolution.week:
        return datetime.timedelta(days=365 * 2)
    elif interval == tinvest.schemas.CandleResolution.month:
        return datetime.timedelta(days=365 * 10)
    raise ValueError("unknown interval value {}".format(interval))


def get_candles(client, figi, from_=None, to_=None, *, interval):
    if to_ is None:
        to_ = datetime.now(tz=timezone("Europe/Moscow"))
    result = []
    duration = get_max_duration(interval)
    d2 = to_
    d1 = d2 - duration
    if from_ is not None and d1 < from_:
        d1 = from_
    while True:
        r = client.get_market_candles(figi, d1, d2, interval)
        result.extend(r.payload.candles)
        d2 = d1
        d1 -= duration
        if (from_ is not None and d2 < from_) or len(r.payload.candles) == 0:
            break
    result = pd.DataFrame([json.loads(x.json()) for x in result])
    result["time"] = pd.to_datetime(result["time"]).dt.tz_convert(tz)
    result = result.sort_values("time").reset_index(drop=True)
    result["Adj Close"] = result["c"]
    result["Date"] = pd.to_datetime(result["time"].dt.date)
    result = result.set_index("Date")
    return result[["Adj Close"]]

# ОФЗ 26207
RU000A0JS3W6 SU26207RMFS9 BBG002PD3452
https://www.tinkoff.ru/invest/bonds/SU26207RMFS9/coupons/

# ОФЗ 26209
RU000A0JSMA2 SU26209RMFS5 BBG00386NQK6
https://www.tinkoff.ru/invest/bonds/SU26209RMFS5/coupons/

# ОФЗ 26218
RU000A0JVW48 SU26218RMFS6 BBG00B9PJ7V0
https://www.tinkoff.ru/invest/bonds/SU26218RMFS6/coupons/

# ОФЗ 26211

RU000A0JTJL3 SU26211RMFS1 BBG003TTSBB1
https://www.tinkoff.ru/invest/bonds/SU26211RMFS1/coupons/
# ОФЗ 26212

RU000A0JTK38 SU26212RMFS9 BBG00425VG07
https://www.tinkoff.ru/invest/bonds/SU26212RMFS9/coupons/

In [7]:
bond_list = ["RU000A0JS3W6", "RU000A0JSMA2", "RU000A0JVW48", "RU000A0JTJL3", "RU000A0JTK38"]
bonds[bonds["isin"].isin(bond_list)]

,currency,figi,isin,lot,min_price_increment,name,ticker,type,min_quantity
102,RUB,BBG00B9PJ7V0,RU000A0JVW48,1,0.01,ОФЗ 26218,SU26218RMFS6,Bond,NaN
131,RUB,BBG002PD3452,RU000A0JS3W6,1,0.01,ОФЗ 26207,SU26207RMFS9,Bond,NaN
149,RUB,BBG00425VG07,RU000A0JTK38,1,0.01,ОФЗ 26212,SU26212RMFS9,Bond,NaN
383,RUB,BBG003TTSBB1,RU000A0JTJL3,1,0.01,ОФЗ 26211,SU26211RMFS1,Bond,NaN
602,RUB,BBG00386NQK6,RU000A0JSMA2,1,0.01,ОФЗ 26209,SU26209RMFS5,Bond,NaN


In [8]:
begin_ = datetime.datetime.strptime(begin, "%Y-%m-%d")
end_ = datetime.datetime.strptime(end, "%Y-%m-%d")

In [9]:
path = "./data/{}.csv"
for _, row in bonds[bonds["isin"].isin(bond_list)].iterrows():
    spath = path.format(row["isin"])
    if os.path.exists(spath):
        continue
    df = get_candles(client, row["figi"], begin_, end_, interval=tinvest.schemas.CandleResolution.day)
    df.to_csv(spath)

In [10]:
df = pd.read_html("./data/cbr.html", encoding="utf8")[0]
df = df.droplevel(0, "columns").rename(columns={"Дата": "Date"})
df["Date"] = pd.to_datetime(df["Date"], format="%d.%m.%Y")
df = df.set_index("Date").astype(float)
df.columns = [float(x.replace(",", ".")) for x in df.columns]
df /= 100
df.to_csv("./data/cbr.csv")